# 3.4. Linear Regression Implementation from Scratch

In [ ]:
%matplotlib inline
import torch
from d2l import torch as d2l

In [ ]:
torch.arange(3)

## 3.4.1. Defining the Model

In [ ]:
class LinearRegressionScratch(d2l.Module):  #@save
    """The linear regression model implemented from scratch."""
    def __init__(self, weights_number, learning_rate, sigma=0.01):
        super().__init__()
        self.save_hyperparameters()
        # 权重矩阵，需要求导
        self.w = torch.normal(0, sigma, (weights_number, 1), requires_grad=True)
        # 偏置量，标量
        self.b = torch.zeros(1, requires_grad=True)

In [ ]:
# decorator的这种方式是为了适配jupyter notebook的方式，不至于一个格子里写完一个class的所有代码，而是通过装饰器一点一点往class里添加
@d2l.add_to_class(LinearRegressionScratch)  #@save
def forward(self, X):
    return torch.matmul(X, self.w) + self.b

## 3.4.2. Defining the Loss Function

In [ ]:
@d2l.add_to_class(LinearRegressionScratch) #@Save
def loss(self, y_hat, y):
    l = (y_hat - y) ** 2/2
    return l.mean()

## 3.4.3. Defining the Optimization Algorithm

In [ ]:
class SGD(d2l.HyperParameters):
    """minibatch stochastic gradient descent"""
    def __init__(self, params, learning_rate):
        self.save_hyperparameters()

    def step(self):
        for param in self.params:
            param -= self.learning_rate * param.grad
        print(self.params)

    def zero_grad(self):
        for param in self.params:
            if param.grad is not None:
                param.grad.zero_()

In [ ]:
@d2l.add_to_class(LinearRegressionScratch) #@Save
def configure_optimizers(self):
    return SGD([self.w, self.b], self.learning_rate)

## 3.4.4. Training

In [ ]:
@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_batch(self, batch):
    return batch

@d2l.add_to_class(d2l.Trainer)  #@save
def fit_epoch(self):
    self.model.train()
    for batch in self.train_dataloader:
        loss = self.model.training_step(self.prepare_batch(batch))
        self.optim.zero_grad()
        with torch.no_grad():
            loss.backward()
            if self.gradient_clip_val > 0:  # To be discussed later
                self.clip_gradients(self.gradient_clip_val, self.model)
            self.optim.step()
        self.train_batch_idx += 1
    if self.val_dataloader is None:
        return
    self.model.eval()
    for batch in self.val_dataloader:
        with torch.no_grad():
            self.model.validation_step(self.prepare_batch(batch))
        self.val_batch_idx += 1

In [ ]:
model = LinearRegressionScratch(2, learning_rate=0.03)
data = d2l.SyntheticRegressionData(w=torch.tensor([2, -3.4]), b=4.2)
data.get_dataloader

In [ ]:
trainer = d2l.Trainer(max_epochs=5)
trainer.fit(model, data)

In [ ]:
trainer = d2l.Trainer(max_epochs=5)
trainer.fit(model, data)